In [1]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('data/scraped_data/reviews_Electronics_5.json.gz')

In [2]:
df.shape

(1689188, 9)

In [3]:
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [4]:
df['reviewText_len'] = df['reviewText'].apply(lambda x:len(x))

In [5]:
df['reviewText_len'].mean()

634.201737166023

In [6]:
g = df.groupby('overall')

In [7]:
df['asin'].nunique()

63001

In [8]:
for name, group in g:
    print (name,(group['reviewText_len']).count())

1.0 108725
2.0 82139
3.0 142257
4.0 347041
5.0 1009026


In [9]:
df_meta = getDF('data/scraped_data/meta_Electronics.json.gz')

In [10]:
df_meta.head(3)

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN


In [ ]:
df_comb = df.merge?

In [11]:
df_comb = df.merge

In [12]:
df_comb = df.merge(df_meta, how="left", on="asin")

In [13]:
df_comb.shape

(1689188, 18)

In [14]:
df_comb.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,reviewText_len,imUrl,description,categories,title,price,salesRank,related,brand
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013",805,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010",2175,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010",4607,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


In [15]:
df_sample = df_comb[df_comb['categories'].apply(lambda x:'tv' in (str(x)).lower())]
#df_sample = df_sample[df_sample['brand'].apply(lambda x:'samsung' in (str(x)).lower())]

In [16]:
df_prod = df_sample[['overall','reviewText','title','brand','categories']]

In [17]:
df_prod = df_prod.rename(columns={'overall':'rating', 'reviewText':'reviews','title':'product','categories':'category'})
df_prod.count()

rating      64262
reviews     64262
product     63611
brand       31268
category    64262
dtype: int64

In [39]:
df_prod[df_prod['brand'] == 'Samsung']

,rating,reviews,product,brand,category,bow
444862,5.0,Direct replacement for my Samsung remote that ...,Samsung BN59-00687A Factory Original Remote Co...,Samsung,"[[Electronics, Accessories & Supplies, Audio &...",direct replacement remote lost. difference glo...
444863,5.0,Ordered a replacement and received this altern...,Samsung BN59-00687A Factory Original Remote Co...,Samsung,"[[Electronics, Accessories & Supplies, Audio &...",ordered replacement received alternative remot...
444864,5.0,I have an older model 32&#34; Samsung TV in on...,Samsung BN59-00687A Factory Original Remote Co...,Samsung,"[[Electronics, Accessories & Supplies, Audio &...",older model 32&#34; guest bedrooms. model ln32...
444865,5.0,I have the uncanny knack of losing remotes per...,Samsung BN59-00687A Factory Original Remote Co...,Samsung,"[[Electronics, Accessories & Supplies, Audio &...",uncanny knack losing remotes permanently least...
444866,5.0,Got this by mistake. Actually needed another ...,Samsung BN59-00687A Factory Original Remote Co...,Samsung,"[[Electronics, Accessories & Supplies, Audio &...","mistake. actually needed another model, everyt..."
444867,5.0,Mine was gone... I had a new one and works per...,Samsung BN59-00687A Factory Original Remote Co...,Samsung,"[[Electronics, Accessories & Supplies, Audio &...",mine gone... new works perfectly... i'm happy ...
542117,4.0,"My family and I just love this TV, particularl...",Samsung UN55B6000 55-Inch 1080p 120 Hz LED HDTV,Samsung,"[[Electronics, Television & Video, Televisions...","family love tv, particularly watching sports. ..."
542118,4.0,I've had this TV since Sept. 2009 and love it....,Samsung UN55B6000 55-Inch 1080p 120 Hz LED HDTV,Samsung,"[[Electronics, Television & Video, Televisions...","i've since sept. 2009 love it. easy up, even s..."
542119,5.0,"Pros: Excellent picture clarity, amazing colo...",Samsung UN55B6000 55-Inch 1080p 120 Hz LED HDTV,Samsung,"[[Electronics, Television & Video, Televisions...","pros: excellent clarity, amazing colors, 120hz..."
542120,5.0,"beautfiul tv, worked flawlessly right out the ...",Samsung UN55B6000 55-Inch 1080p 120 Hz LED HDTV,Samsung,"[[Electronics, Television & Video, Televisions...","beautfiul tv, worked flawlessly right box. can..."


In [18]:
from src.naive_review_analyzer import NaiveReviewAnalyzer
nra = NaiveReviewAnalyzer()
nra.create_bow(df_prod)

# for name, group in df_prod.groupby('product'):
#     print(name, '\n\t',nra.create_word_list(product=name, rating=[5.0]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bhaskarballapragada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bhaskarballapragada/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
import ipywidgets
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [21]:
from src.LdaReviewAnalyzer import LdaReviewAnalyzer
lda = LdaReviewAnalyzer(num_topics=5)
lda.build_vectorize(df_prod)
lda.fit(df_prod, rating=[5])
lda.transform()
topics_dict = lda.get_topics()


/anaconda3/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [22]:
print (topics_dict)

{'Topic 0:': [('would', 5300.158212403265), ('like', 5282.252313740159), ('good', 5045.357059550335), ('price', 4672.704170510319), ('get', 4427.02868046432), ('really', 4051.7575958682855), ('well', 4036.568083135732), ('smart', 3488.733400902889), ('buy', 3302.4942106337057), ('works', 3285.196633491496)], 'Topic 1:': [('antenna', 10912.485909226147), ('channels', 8085.649643923279), ('cable', 6581.637832312293), ('get', 3967.363126317774), ('signal', 3893.334433241468), ('stations', 2579.2760667453376), ('digital', 2171.439119653906), ('reception', 2006.8055614095251), ('local', 1810.5481909990851), ('channel', 1774.5743551758446)], 'Topic 2:': [('mount', 10947.629996696167), ('wall', 8927.870948624342), ('easy', 5132.73864743966), ('would', 3010.9979333188444), ('well', 2878.188525087894), ('install', 2849.5555573843862), ('price', 2437.7119428302894), ('mounting', 2299.225657565528), ('mounted', 2214.4119562251803), ('product', 2164.3658917856746)], 'Topic 3:': [('sound', 11224.48

In [35]:
lda.save_topic_model('src/templates/lda.html')

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [ ]:
for name, group in df_prod.groupby('product'):
    count = group.count()
    if (count[1] > 100):
        lda = LdaReviewAnalyzer(df_prod)
        lda.fit_transform(num_topics=2, rating=[0,1])
        topics_dict = lda.get_topics()
        lda.display_topic_model()
        print (name)
        print (topics_dict)
        print ("____________")

In [ ]:
from src.w2v_review_analyzer import W2VReviewAnalyzer
import pandas as pd
df = pd.DataFrame()
w2v = W2VReviewAnalyzer(df)
# w2v.fit()

In [ ]:
import pickle
#pickle.dump( w2v.model, open( f'static/w2v.pkl', "wb" ) )
w2v.model = pickle.load(open( f'static/w2v.pkl', "rb" ) )

In [ ]:
w2v.model.wv.vocab


In [ ]:
#w2v.model.wv.most_similar(positive=['samsung',['screen'])

samsung_vec = w2v.model.wv.get_vector('samsung')
broken_vec = w2v.model.wv.get_vector('broken')

w2v.model.wv.most_similar(positive=['ln46c630','cost'])

#w2v.model.wv.similarity('defective','samsung')

In [ ]:
import tensorflow as tf
import os
from tensorflow.contrib.tensorboard.plugins import projector
import warnings
warnings.filterwarnings("ignore")

def visualize(model, output_path):
    meta_file = "w2x_metadata.tsv"
    placeholder = np.zeros((len(model.wv.index2word), 100))

    with open(os.path.join(output_path,meta_file), 'wb') as file_metadata:
        for i, word in enumerate(model.wv.index2word):
            placeholder[i] = model[word]
            # temporary solution for https://github.com/tensorflow/tensorflow/issues/9094
            if word == '':
                print("Emply Line, should replecaed by any thing else, or will cause a bug of tensorboard")
                file_metadata.write("{0}".format('<Empty Line>').encode('utf-8') + b'\n')
            else:
                file_metadata.write("{0}".format(word).encode('utf-8') + b'\n')

    # define the model without training
    sess = tf.InteractiveSession()

    embedding = tf.Variable(placeholder, trainable = False, name = 'w2x_metadata')
    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(output_path, sess.graph)

    # adding into projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = 'w2x_metadata'
    embed.metadata_path = meta_file

    # Specify the width and height of a single thumbnail.
    projector.visualize_embeddings(writer, config)
    saver.save(sess, os.path.join(output_path,'w2x_metadata.ckpt'))
    print('Run `tensorboard --logdir={0}` to run visualize result on tensorboard'.format(output_path))

In [ ]:

visualize(w2v.model.wv, 'data/tensor/')

In [ ]:
l = [{'a':4},{'b':2},{'c':3}]

In [ ]:
sorted(l)